Goal: 
(1) CDC Covid case and death data are daily updated, the most recent dataset we get if on 11/27/2020.
(2) yelp temporary closed dataset is updated on 6/10/2020
(3) We would like to predict yelp temporary closed information in November 2020 with CDC dataset and the current yelp dataset

In [1]:
import pandas as pd
import os

In [2]:
os.chdir('/Documents/MSIS/Fall 2020/Team Project')
os.getcwd()

'/Documents/MSIS/Fall 2020/Team Project'

In [3]:
caseCDC = pd.read_csv('caseCDC.csv')
caseCDC.head()

,submission_date,state,tot_cases,new_case,tot_death,new_death
0,2020-01-22,CO,0,0,0,0
1,2020-01-23,CO,0,0,0,0
2,2020-01-24,CO,0,0,0,0
3,2020-01-25,CO,0,0,0,0
4,2020-01-26,CO,0,0,0,0


In [4]:
#get CDC training dataset on 6/10/2020
x = caseCDC[caseCDC['submission_date']=='2020-06-10'][["state", "tot_cases", "tot_death"]]
print(len(x))
x.set_index(['state'], inplace = True)
x.head()

60


,tot_cases,tot_death
state,,
CO,28499,1573
FL,65779,2801
AZ,29852,1095
SC,15759,575
CT,44347,4120


In [5]:
#get CDC testing dataset on 11/27/2020
test_x = caseCDC[caseCDC['submission_date']=='2020-11-27'][["state", "tot_cases", "tot_death"]]
print(len(test_x))
test_x.set_index(['state'], inplace = True)
test_x.head()

60


,tot_cases,tot_death
state,,
CO,220953,2977
FL,963751,18363
AZ,318638,6588
SC,213120,4346
CT,112581,4961


In [6]:
#get yelp dataset on 6/10/2020
yelpState = pd.read_csv('yelpState.csv')
yelpState.head()

,business_id,CovidBanner,TemporaryClosedUntil,IsOpen,State
0,--1UhMGODdWsrMastO9DZw,FALSE,FALSE,1,AB
1,--6MefnULPED_I942VcFNA,FALSE,FALSE,1,ON
2,--7zmmkVg-IMGaXbuVd0SQ,FALSE,FALSE,1,NC
3,--8LPVSo5i0Oo61X01sV9A,FALSE,FALSE,1,AZ
4,--9QQLMTbFzLJ_oT-ON3Xw,FALSE,FALSE,1,AZ


In [7]:
#aggregate yelp dataset on state level
businessCnt = yelpState[["business_id", "State"]].groupby('State').count()
bannerCnt = yelpState[yelpState['CovidBanner'] != 'FALSE'][["CovidBanner", "State"]].groupby('State').count()
TemporaryClosedCnt = yelpState[yelpState['TemporaryClosedUntil'] != 'FALSE'][["TemporaryClosedUntil", "State"]].groupby('State').count()

In [8]:
yelp = businessCnt.join(bannerCnt).join(TemporaryClosedCnt).fillna(0)
yelp['BannerPct'] = yelp['CovidBanner']/yelp['business_id']
yelp['TempClosedPct'] = yelp['TemporaryClosedUntil']/yelp['business_id']
yelp.head()

,business_id,CovidBanner,TemporaryClosedUntil,BannerPct,TempClosedPct
State,,,,,
AB,8685,202.0,31.0,0.023258,0.003569
AK,1,0.0,0.0,0.000000,0.000000
AL,3,0.0,0.0,0.000000,0.000000
AR,1,0.0,0.0,0.000000,0.000000
AZ,60987,5643.0,232.0,0.092528,0.003804


In [9]:
# some state in yelp like 'AB' which is a state in Canada are eliminated
# also yelp doesn't have all states's data in US
# we get the states' data which are from both CDC and yelp
#combine covid and yelp dataset for training
data_train = x.join(yelp, how='inner').fillna(0)  
print(len(data_train))
data_train.head()

27


,tot_cases,tot_death,business_id,CovidBanner,TemporaryClosedUntil,BannerPct,TempClosedPct
CO,28499,1573,2,2.0,0.0,1.000000,0.000000
FL,65779,2801,3,1.0,0.0,0.333333,0.000000
AZ,29852,1095,60987,5643.0,232.0,0.092528,0.003804
SC,15759,575,1329,104.0,4.0,0.078254,0.003010
CT,44347,4120,2,0.0,0.0,0.000000,0.000000


In [10]:
data_test = test_x.join(yelp[['business_id']], how = 'inner').fillna(0)
print(len(data_test))
data_test.head()

27


,tot_cases,tot_death,business_id
CO,220953,2977,2
FL,963751,18363,3
AZ,318638,6588,60987
SC,213120,4346,1329
CT,112581,4961,2


built a model for predit BannerPct

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
#from sklearn.decomposition import PCA

In [12]:
#try to find the best kernel for SVR{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}
model = SVR(gamma='auto', kernel='linear')
reg = model.fit(data_train[["tot_cases", "tot_death", "business_id"]], data_train["BannerPct"])
mse = mean_squared_error(data_train["BannerPct"], reg.predict(data_train[["tot_cases", "tot_death", "business_id"]]))
print("Training MSE: " + str(mse))

Training MSE: 26620.772192021523


In [13]:
# We tried 4 different model and would like to pick the best model for the later pediction
train_X = data_train[["tot_cases", "tot_death", "business_id"]]
train_Y = data_train["BannerPct"]
models = [
            ('LinearRegression', LinearRegression()),
            ('DecisionTreeRegressor', DecisionTreeRegressor()),
            ('SupportVectorRegression', SVR(gamma='auto', kernel='rbf')),
            ('KNearestNeighborRegression', KNeighborsRegressor())
        ]
print("Training MSE: ")
for name, model in models:
    reg = model.fit(train_X, train_Y)
    mse = mean_squared_error(train_Y, reg.predict(train_X))
    print(name + " " + str(mse))            

Training MSE: 
LinearRegression 0.11192415212927687
DecisionTreeRegressor 0.0
SupportVectorRegression 0.009872676590296512
KNearestNeighborRegression 0.10325458350329175


predict BannerPct for States in November 2020

In [14]:
#DecisionTreeRegressor() is our best model
model = DecisionTreeRegressor()
reg = model.fit(train_X, train_Y)
BannerPct = reg.predict(data_test)


In [15]:
#get predicted data for Banner count : how many restaurants will post banner in yelp in November 2020
data_test['BannerPct'] = BannerPct
data_test['CovidBanner'] = [round(num) for num in data_test['BannerPct'] * data_test['business_id']] 
data_test.head()

,tot_cases,tot_death,business_id,BannerPct,CovidBanner
CO,220953,2977,2,0.333333,1
FL,963751,18363,3,0.333333,1
AZ,318638,6588,60987,0.130435,7955
SC,213120,4346,1329,0.130435,173
CT,112581,4961,2,0.333333,1


built model for TemporaryClosedUntil

In [16]:
train_X = data_train[["tot_cases", "tot_death", "business_id"]]
train_Y = data_train["TempClosedPct"]
models = [
            ('LinearRegression', LinearRegression()),
            ('DecisionTreeRegressor', DecisionTreeRegressor()),
            ('SupportVectorRegression', SVR(gamma='auto', kernel='rbf')),
            ('KNearestNeighborRegression', KNeighborsRegressor())
        ]
print("Training MSE: ")
for name, model in models:
    reg = model.fit(train_X, train_Y)
    mse = mean_squared_error(train_Y, reg.predict(train_X))
    print(name + " " + str(mse))  

Training MSE: 
LinearRegression 5.9465905588468676e-06
DecisionTreeRegressor 1.247588861696376e-08
SupportVectorRegression 4.909339423846224e-05
KNearestNeighborRegression 5.794836065118636e-06


In [17]:
#predict Temporary Closed in November 2020
model = DecisionTreeRegressor()
reg = model.fit(train_X, train_Y)
ClosedPct = reg.predict(data_test[["tot_cases", "tot_death", "business_id"]])

In [18]:
data_test['TempClosedPct'] = ClosedPct
data_test['TemporaryClosedUntil'] = [round(num) for num in data_test['TempClosedPct'] * data_test['business_id']] 
data_test

,tot_cases,tot_death,business_id,BannerPct,CovidBanner,TempClosedPct,TemporaryClosedUntil
CO,220953,2977,2,0.333333,1,0.000000,0
FL,963751,18363,3,0.333333,1,0.000000,0
AZ,318638,6588,60987,0.130435,7955,0.003518,215
SC,213120,4346,1329,0.130435,173,0.002976,4
CT,112581,4961,2,0.333333,1,0.000000,0
NE,124066,984,2,0.000000,0,0.000000,0
WA,158167,2703,5,0.000000,0,0.000000,0
PA,343614,10234,12390,0.130435,1616,0.006295,78
OH,399808,6346,16402,0.130435,2139,0.003518,58
VA,233618,4054,1,1.000000,1,0.000000,0


Discuss:
(1) Politic could affect the Covid Close
(2) Vacinne program could also affect the Covid Close